# Step 4 - Create a new dataset to share with coworkers

In the previous section we created a query that returns all the data we wanted. It is now time to create a new dataset from the result of this query, which allows us and our coworkers to make us of this new data. An example could be to visualize it as a bike suitability map in QGIS.

We will explore the following options how to achieve this:
- Create a table from the results.
- Create a view from the results.
- Create a materialized view from the results.

***
# Create a table from the results
Creating a new table from the results of a query is as easy as adding `CREATE TABLE <NEW_TABLE_NAME> AS` to the beginning of the query and executing it in the usual way. After refreshing the tables section (in pgAdmin right-click _Tables_ and select refresh) you will see a new table. The new table is independent from the data it was derived from and thus it does not change when the datasets that it was derived from change. The query used to create the new table is only used during the table creation and not stored as part of the table.

```postgresql
CREATE TABLE t_bike_suitability AS

WITH sum_total_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total_table
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid
),
sum_bike_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
        AND zh_roads.velo=1
    GROUP BY zh_districts.fid
)

SELECT
    zh_districts.fid AS district_id,
    zh_districts.stzname AS zonenname, 
    sum_total_table.sum_total_table AS sum_total,
    sum_bike_table.sum_bike AS sum_bike,
    sum_bike_table.sum_bike/sum_total_table.sum_total_table AS ratio_suitable,
    zh_districts.geom AS geom
FROM zh_districts
LEFT JOIN sum_total_table ON sum_total_table.fid = zh_districts.fid
LEFT JOIN sum_bike_table ON sum_bike_table.fid = zh_districts.fid;
```

***
# Create a view from the results.
If you think of a table as something that stores data, think of a view as something that provides you a specific view on your data. A view can be thought of as a virtual table with a query at its core that provides the content. A view looks like a table and you interact with it in the same way using your everyday SQL clauses, but the content of the view is always generated ad hoc at the time of the interaction and is the result of whatever the underlying query returns.

Views can be created by adding `CREATE VIEW <NEW_VIEW_NAME> AS` at the beginning of your query and executing it the usual way. Once created views can be found under _Views_ in the pgAdmin database menu (same place where the _Tables_ folder is located). The query at the heart of a view can be found by right-clicking a view under properties>code.


```postgresql
CREATE VIEW v_bike_suitability AS

WITH sum_total_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total_table
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid
),
sum_bike_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
        AND zh_roads.velo=1
    GROUP BY zh_districts.fid
)

SELECT
    zh_districts.fid AS district_id,
    zh_districts.stzname AS zonenname, 
    sum_total_table.sum_total_table AS sum_total,
    sum_bike_table.sum_bike AS sum_bike,
    sum_bike_table.sum_bike/sum_total_table.sum_total_table AS ratio_suitable,
    zh_districts.geom AS geom
FROM zh_districts
LEFT JOIN sum_total_table ON sum_total_table.fid = zh_districts.fid
LEFT JOIN sum_bike_table ON sum_bike_table.fid = zh_districts.fid;
```

***
# Create a materialized view from the results.
Similar to views, a materialized view has a query at its heart which determines its content. In contrast to a view, a materialized view executes this query only when explicitly triggered and "materializes" the results into a table. This procedure is what is called "refreshing a materialized view", think of it as caching the results of the query. Materialized views are particularly useful if the underlying query is complex and takes a substantial amount of time to run. Interacting with a materialized view is as fast as working with a normal table, no matter how complex the underlying query is and a simple refresh from time to time allows to update its content based on the underlying datasets used by the query. Compared to a normal view the downsides are of course that an explicit periodic refresh has to be scheduled, which adds complexity to the setup, and that data might become outdated between refreshes. For the interested reader I recommend [this article](https://www.compose.com/articles/its-a-view-its-a-table-no-its-a-materialized-view/).

Materialized views can be created by adding `CREATE MATERIALIZED VIEW <NEW_VIEW_NAME> AS` at the beginning of your query and executing it the usual way. Once created materialized views can be found under _Materialized Views_ in the pgAdmin database menu (same place where the _Tables_ folder is located). The query at the heart of a materialized view can be found by right-clicking a view under properties>definitions. Refreshing a materialized view is possible by executing `REFRESH MATERIALIZED VIEW <VIEW_NAME>;`

```postgresql
CREATE MATERIALIZED VIEW mv_bike_suitability AS

WITH sum_total_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_total_table
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid
),
sum_bike_table AS (
    SELECT zh_districts.fid, 
        SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS sum_bike
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
        AND zh_roads.velo=1
    GROUP BY zh_districts.fid
)

SELECT
    zh_districts.fid AS district_id,
    zh_districts.stzname AS zonenname, 
    sum_total_table.sum_total_table AS sum_total,
    sum_bike_table.sum_bike AS sum_bike,
    sum_bike_table.sum_bike/sum_total_table.sum_total_table AS ratio_suitable,
    zh_districts.geom AS geom
FROM zh_districts
LEFT JOIN sum_total_table ON sum_total_table.fid = zh_districts.fid
LEFT JOIN sum_bike_table ON sum_bike_table.fid = zh_districts.fid;
```

***

**Your turn:**
- Execute all three queries to create a table, a view and a materialized view from the query result. 
- Recall the goals and requirements stated in the introduction session and come up with a suggestion which of the three approaches to use.
- For your suggested approach, sketch out the steps to update your new dataset on a monthly basis.